In [1]:
!pip install requests azure-functions
!pip install openai
!pip install azure-ai-contentsafety

In [ ]:
import requests
import json
import logging
import azure.functions as func

from azure.core.credentials import AzureKeyCredential
from azure.ai.textanalytics import TextAnalyticsClient

from openai import AzureOpenAI

from azure.ai.contentsafety import ContentSafetyClient
from azure.core.credentials import AzureKeyCredential
from azure.core.exceptions import HttpResponseError
from azure.ai.contentsafety.models import AnalyzeTextOptions, TextCategory


In [ ]:
# Variables de entorno con las claves de los servicios
AZURE_LANGUAGE_API_URL = ''  # Cambiar <tu-endpoint> al URL correcto de tu recurso
AZURE_LANGUAGE_API_KEY = ''  # Cambia por tu clave de suscripción

AZURE_OPENAI_API_URL = ''  # Cambiar <tu-endpoint-openai>
AZURE_OPENAI_API_KEY = ''  # Cambiar clave de OpenAI

AZURE_CONTENT_SAFETY_API_URL = ''
AZURE_CONTENT_SAFETY_API_KEY = ''


In [6]:


# Función para analizar texto con Azure Language
def analyze_text(prompt):
    """ Analiza el texto usando Azure Language Services. """
    try:
        text_analytics_client = TextAnalyticsClient(endpoint=AZURE_LANGUAGE_API_URL, credential=AzureKeyCredential(AZURE_LANGUAGE_API_KEY))
        response = text_analytics_client.recognize_entities(documents=[prompt])

        if response:
            return response
        else:
            logging.error("Error en la API: No se recibieron datos.")
            return {"error": "No se recibieron datos de la API."}
    except Exception as e:
        logging.error(f"Error en la API: {str(e)}")
        return {"error": f"Falló la solicitud: {str(e)}"}

# Uso de Azure OpenAI para reescribir texto
def rewrite_prompt_with_openai(prompt):
    endpoint = AZURE_OPENAI_API_URL
    model_name = "gpt-35-turbo"
    deployment = "gpt-35-turbo"
    subscription_key = AZURE_OPENAI_API_KEY
    api_version = "2024-12-01-preview"

    client = AzureOpenAI(
        api_version=api_version,
        azure_endpoint=endpoint,
        #azure_ad_token_provider=token_provider,
        api_key=subscription_key,
    )

    # Solicitud de reescritura de texto
    try:
        response = client.chat.completions.create(
            messages=[
                {"role": "system", "content": "Corrige la gramática del siguiente texto y mejora su claridad."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=4096,
            temperature=1.0,
            top_p=1.0,
            model=deployment
        )
        return response.choices[0].message.content
    except Exception as e:
        logging.error(f"Error en OpenAI API: {str(e)}")
        return {"error": f"Falló la solicitud: {str(e)}"}

# Verificación de seguridad de contenido
def check_content_safety(prompt):
    """ Evalúa contenido potencialmente dañino con Azure Content Safety. """
    # analyze text
    key = AZURE_CONTENT_SAFETY_API_KEY
    endpoint =  AZURE_CONTENT_SAFETY_API_URL

    # Create an Azure AI Content Safety client
    client = ContentSafetyClient(endpoint, AzureKeyCredential(key))

    # Contruct request
    request = AnalyzeTextOptions(text=prompt)

    # Analyze text
    try:
        response = client.analyze_text(request)
    except HttpResponseError as e:
        print("Analyze text failed.")
        if e.error:
            print(f"Error code: {e.error.code}")
            print(f"Error message: {e.error.message}")
            raise
        print(e)
        raise

    hate_result = next(item for item in response.categories_analysis if item.category == TextCategory.HATE)
    self_harm_result = next(item for item in response.categories_analysis if item.category == TextCategory.SELF_HARM)
    sexual_result = next(item for item in response.categories_analysis if item.category == TextCategory.SEXUAL)
    violence_result = next(item for item in response.categories_analysis if item.category == TextCategory.VIOLENCE)

    if hate_result:
        print(f"Hate severity: {hate_result.severity}")
    if self_harm_result:
        print(f"SelfHarm severity: {self_harm_result.severity}")
    if sexual_result:
        print(f"Sexual severity: {sexual_result.severity}")
    if violence_result:
        print(f"Violence severity: {violence_result.severity}")

# Función principal
def main(req: func.HttpRequest) -> func.HttpResponse:
    """ Procesa un prompt y lo analiza con los servicios de Azure. """
    logging.info("Procesando solicitud...")

    try:
        req_body = req.get_json()
        prompt = req_body.get("prompt", "")

        if not prompt:
            return func.HttpResponse("Falta el parámetro 'prompt'", status_code=400)

        # 1. Análisis de texto
        analysis_result = analyze_text(prompt)

        # 2. Reescribir texto
        corrected_prompt = rewrite_prompt_with_openai(prompt)

        # 3. Evaluación de contenido
        safety_result = check_content_safety(corrected_prompt)

        # Respuesta final
        response_data = {
            "originalPrompt": prompt,
            "correctedPrompt": corrected_prompt,
            "textAnalysis": analysis_result,
            "contentSafety": safety_result
        }

        return func.HttpResponse(
            body=json.dumps(response_data, indent=2),
            mimetype="application/json",
            status_code=200
        )

    except Exception as e:
        logging.error(f"Error en el procesamiento: {str(e)}")
        return func.HttpResponse(f"Error interno: {str(e)}", status_code=500)

In [ ]:
#Funcion de prueba para validar las conexiones
def test_function():
    """Realiza una prueba básica de las funcionalidades implementadas."""
    # Prompt de prueba
    prompt = "Hola, este texto necdesita ser analizado y mejorado."
    print(f"\nPrompt de prueba: {prompt}")
    # 1️⃣ Análisis de texto
    print("\n🔍 1️⃣ Analizando texto...")
    #prompt = "Tu texto aquí"  # Asegúrate de definir el texto que deseas analizar
    analysis_result = analyze_text(prompt)
    print(analysis_result)
    #print(json.dumps(analysis_result, indent=2))

    # 2️⃣ Reescritura del texto
    print("\n✍️ 2️⃣ Reescribiendo el prompt...")
    corrected_prompt = rewrite_prompt_with_openai(prompt)
    print(corrected_prompt)

    # 3️⃣ Evaluación de seguridad del contenido
    print("\n🛡 3️⃣ Evaluando seguridad del contenido...")
    safety_result = check_content_safety(corrected_prompt)
    print(safety_result)
    #print(json.dumps(safety_result, indent=2))

    # 4️⃣ Respuesta final
    print("\n✅ 4️⃣ Respuesta final:")
    response_data = {
        "originalPrompt": prompt,
        "correctedPrompt": corrected_prompt,
        "textAnalysis": analysis_result,
        "contentSafety": safety_result
    }
    print(response_data)

# Llama a la función
test_function()


Prompt de prueba: Hola, este texto necdesita ser analizado y mejorado.

🔍 1️⃣ Analizando texto...
[RecognizeEntitiesResult(id=0, entities=[CategorizedEntity(text=analizado, category=Skill, subcategory=None, length=9, offset=31, confidence_score=0.66)], warnings=[], statistics=None, is_error=False, kind=EntityRecognition)]

✍️ 2️⃣ Reescribiendo el prompt...
Hola, este texto necesita ser analizado y mejorado.

🛡 3️⃣ Evaluando seguridad del contenido...
Hate severity: 0
SelfHarm severity: 0
Sexual severity: 0
Violence severity: 0
None

✅ 4️⃣ Respuesta final:
{'originalPrompt': 'Hola, este texto necdesita ser analizado y mejorado.', 'correctedPrompt': 'Hola, este texto necesita ser analizado y mejorado.', 'textAnalysis': [RecognizeEntitiesResult(id=0, entities=[CategorizedEntity(text=analizado, category=Skill, subcategory=None, length=9, offset=31, confidence_score=0.66)], warnings=[], statistics=None, is_error=False, kind=EntityRecognition)], 'contentSafety': None}
